# Analysis of Single Cell RNA Sequencing 

DZ, 27.01.2023

Please note that this tutorial is strongly simplified to provide a brief overview about sequential RNAseq analysis steps.

In [1]:
# Warnings should be always considered and therefore only turned off exceptionally.
# The improvement of the readability of this tutorial might be such an exception ;) 
import warnings
warnings.filterwarnings('ignore')

### Step 1: Load data and understand the structure
<ol>
    <li>Import the raw count table as pandas dataframe, therefore import the pandas package first. It's quite typical to name it pd.</li>
    <li>Use the pandas function read_csv </li>
    <li> Once the csv file was read succesfully, you can display its content. Be careful. These files can be quite large. <br/>Therefore, to get an idea of the size of the input data one can use the shape attrbiute
</ol>



To understand what the data look like it's usually enough to display the first 10 rows. Print the first 10 rows of the table using the head() function. 

It is also possible to see the last 10 rows by using the tail function. Print the last 10 rows of the table using the tail() function

A very typical characteristic of the single cell sequencing results is the non-normal distribution of gene counts per cell. <br/>
To display the data distribution, we will count the number of gene counts per cell and store them in a list.<br/>
<ol>
    <li> Create a list with sums of all row values per column (== gene counts per cell) using .sum().values </li>
    <li> To get the maximum and minimum within a list, numpy comes with implemented functions. Therefore we have to import numpy too.</li>
     <li>  Print the maximum and the minimum count within the new created list using np.max() and np.min().</li>
<li> To finally plot the histogramm, we can use seaborn plotting package which comes with this specific histogramm function sns.histplot(). <br/> Therefore, import seaborn as sns and use the histplot() function. </li> </ol>

## 2. Preprocess Your Data
We start the preprocessing our data by keeping only cells with a biological reasonable amount of gene counts 500-3500, all other cells will be removed.
If you modify the raw data it's quite clever to work with a copy to avoid reloading of the .csv files if you make an unintented mistake (happens to the best ;) ).

<ol>
    <li>create variables minimum_number_of_counts and maximum_number_of_counts which should hold the manual selected thresholds</li> 
    <li> copy the dataframe into a new dataframe</li>
    <li> write a for loop to go through the df column-wise. check if the count of the column exceeds the threshold. <br/>Remove the column by using the drop() function. Make sure to overwrite the df.</li>
</ol>


In [2]:
#count_adjusted_data = data.copy()

#for column in count_adjusted_data.columns:
#        if (count_adjusted_data[column].sum() > maximum_number_of_counts) or (count_adjusted_data[column].sum() < minimum_number_of_counts) :
#            count_adjusted_data = count_adjusted_data.drop(column, axis = 1)
            
#print("Number of cells in the raw data: ", data.shape[1])
#print("Number of cells in the processed data: ", count_adjusted_data.shape[1])



Lets see how the data distribution looks like after cells were removed from the analysis. Plot again the data distribution as done before with the sns.histplot() function.

### Normalization is very important !
Data Normalization is very important for all further downstream analysis and compromises two steps. <br/> 
(1): To make counts comparable among cells, we need to normalize for the total count. This is also called library-size-correction. Normalize each cell by total counts over all genes (sum), so that every cell has the same total count after normalization. <br/>
(2) In later steps we will apply different mathematical functions that require normal distribution. Therefore log-transform our data in this step. Log- transformation serves as a variance stabilisation and approximates a normal distribution.
One can do log-normalization by hand as follows:

In [3]:

#data_log = count_adjusted_data.apply(lambda x: np.log(((x/sum(x))*10000)+1))
#sns.histplot((data_log.sum()))

## The Scanpy Package
So far, we just handled a basic dataframe from a csv file. This was not RNAseq specific.
When it comes to more RNAseq specific analysis, similar to Seurat Package in R, one can use scanpy package in python. It provides many ready-to-use implemented functions for RNAseq analysis. Scanpy works with a so called annotation data object (anndata). This can be easily created from our pandas raw data frame.

<ol>
    <li> load the scanpy package</li>
    <li>load the anndata package</li>
    <li>create a new anndata object called 'scanpy_object' using the function ad.AnnData()</li>

Instead of calculating the gene counts per cell using the sum() function, scanpy allows to quickly plot for genes with the highest expression. 

Use the scanpy plot function highest_expr_genes() and plot the top 20 genes with the function sc.pl.highest_expr_genes().

In parallel to previous filtering steps, scanpy allows data filtering too.

<ol>
    <li> Use scanpy preprocessing function sc.pp.filter_cells() and only keep cells with more than 500 (min_counts) genes and less than 3500 (max_counts) genes</li> <li>  double check the filtering results. are there differents ? </li>
</ol>


Use the scanpy preprocessing pipeline to normalize logscale the data and replot the data distribution

In [4]:
#sc.pp.normalize_total(scanpy_object, target_sum=1e4)
#sc.pp.log1p(scanpy_object)
#count_list = scanpy_object.to_df().T.sum()
#sns.histplot(count_list)

## 3. Dimensional Reduction Methods

In this tutorial, we will compare linear principal component analysis (PCA) and non-linear t-SNE dimensional reduction method results.

### PCA: 
Principal component analysis compromises data standardization, covariance matrix calculation, eigenvector and eigenvalue determination, principal component selection and finally the transformation of the input data. 
Scanpy already provides a read to use pca function sc.pp.pca(). <br/> 
<ol>
    <li> Scale your data first using sc.pp.scale() </li>
    <li> Run PCA on your scanpy object using svd_solver='arpack' using sc.tl.pca().  </li>
</ol>


Visualize the different components using sc.pl.pca( ... , components = ['1,2','3,4','5,6','7,8'], ncols=2)

Plot the Variance Ratio using sc.pl.pca_variance_ratio(... , log=True, n_pcs = 50)

### t-SNE
Create a t-SNE plot using scanpy using sc.tl.tsne(). You could try to measure the time by importing time package and use time.time()

Plot the t-SNE graph and view the changes in the AnnData object using sc.pl.tsne(...)

### UMAP
Create a UMAP using sc.tl.umap(). Therefore, you have to create the neighborhood graph first using sc.pp.neighbors().

Plot the UMAP using sc.pl.umap()

### Cluster and Compare the Plots
Since we already calculated the neighborhood graph, we can apply a clustering algorithm to identify related points (cells) and compare the results with our 3 dimensional reduction methods. 
<ol>
    <li>Use sc.tl.leiden() to cluster your data  </li>
    <li>Plot PCA results using sc.pl.pca  </li>
    <li>Plot t-SNE results using sc.pl.tsne()</li>
    <li>Plot UMAP using sc.pl.umap() </li>
    
</ol>
